# Projekt 1 z przedmiotu DS&CC
Przeanalizowanie zbioru danych, dotyczących wynajmu lokali poprzez platformę **Airbnb**.
W moim przypadku wykorzystane są dane dla miasta ***Neapol, Kampania, Włochy***.

### Krok 0 - Importowanie niezbędnych modułów do dalszej pracy

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt

### Krok 1 - Załadowanie
Poprawne załadowanie danych ze źródła internetowego do ramki danych (*DataFrame*).

In [ ]:
# Warto także pamiętać o poprawnym wpisaniu adresu URL
url = "http://data.insideairbnb.com/italy/campania/naples/2023-09-13/data/listings.csv.gz"

Jak można zauważyć na stronie z danymi oraz w adresie URL, plik z danymi CSV jest pobierany w formacie `.gz`.
Żeby poprawnie wczytać i rozpakować dane, używam parametru `compression='gzip'`. Do określenia kodowania danych używam parametru `encoding='utf-8'`. Jednak w tym przypadku wystarczy wykorzystanie funkcji domyślnej bezargumentowej. Pandas automatycznie wykrywa nagłówki, jeżeli dane ich zawierają.

In [ ]:
df = pd.read_csv(url)

# zaladowanie z okreslonymi parametrami
# df = pd.read_csv(url, compression='gzip', encoding='utf-8')

### Krok 2 - Poznanie
Poznanie rozmiaru zbioru danych (liczby obserwacji i liczby zmiennych, które je opisują) i oszacowanie czasochłonności procesu analizy.

Za pomocą funkcji `df.shape` można dostać krotkę reprezentującą wymiarowość ramki ***DataFrame***.

In [ ]:
df.shape

W przypadku tego zbioru danych dostałem dwuelementową krotkę, w której *0*-wy element reprezentuje wierszy (liczba obserwacji) i *1*-szy element reprezentuje kolumny (liczba zmiennych). 

Prostymi poleceniami poznaję liczbę obserwacji, liczbę zmiennych oraz całkowity rozmiar zbioru danych:

In [ ]:
liczba_obserwacji = df.shape[0]
liczba_zmiennych = df.shape[1]
print("Liczba obserwacji:", liczba_obserwacji, "\nLiczba zmiennych:", liczba_zmiennych)

rozmiar_zbioru = df.size
print("\nRozmiar zbioru danych:", rozmiar_zbioru)

df.info()

### Krok 3 - Wyświetlenie
Wyświetlenie próbki surowych danych w celu wyrobienia sobie wyobrażenia o nich – poznania struktury danych i wstępnej oceny przydatności poszczególnych zmiennych.

Po pobraniu danych ze źródła należy sprawdzić, czy dane są poprawnie wyświetlane oraz zapoznać się z ich strukturą.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Krok 4 - Weryfikacja
Weryfikacja typów poszczególnych zmiennych (całkowite, zmiennoprzecinkowe, kategoryczne porządkowe, kategoryczne nominalne, zmienne typu logicznego, daty) i ich ewentualna korekta (zamiana typu string na float, interpretacja zmiennych numerycznych jako kategorii itp.).

In [ ]:
# zmiana wartości str na float
df['host_response_rate'] = df['host_response_rate'].str.replace('[\%]', "", regex=True).astype(float)/100
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('[\%]', "", regex=True).astype(float)/100
df['price'] = df['price'].str.replace('[\$,]', "", regex=True).astype(float)

# zmiana wartości str na bool
df.host_is_superhost = df.host_is_superhost.map({'f': False, 't': True})
df.host_has_profile_pic = df.host_has_profile_pic.map({'f': False, 't': True})
df.has_availability = df.has_availability.map({'f': False, 't': True})
df.host_identity_verified = df.host_identity_verified.map({'f': False, 't': True})
df.instant_bookable = df.instant_bookable.map({'f': False, 't': True})

# ustawienie zmiennych kategorycznych
df.host_response_time = df.host_response_time.astype('category')
df.source = df.source.astype('category')
df.host_location = df.host_location.astype('category')
df.property_type = df.property_type.astype('category')
df.room_type = df.room_type.astype('category')
df.neighbourhood = df.neighbourhood.astype('category')
df.neighbourhood_cleansed = df.neighbourhood_cleansed.astype('category')
df.host_neighbourhood = df.host_neighbourhood.astype('category')

# podział stringa na listę wartości
df.host_verifications = df.host_verifications.str[1:-1].replace("[' ]", '', regex=True).str.split(',')

# użycie json.loads do ustawienia wartości kodowanych jsonem
df.amenities = df.amenities.apply(json.loads)

# ustawienie liczby łazienek na typ float
df.bathrooms = df.bathrooms_text.str.replace("-", "0.5").str.replace("[a-zA-Z ]", "", regex=True).astype(float)

# parsing daty
df.host_since = pd.to_datetime(df.host_since)
df.calendar_last_scraped = pd.to_datetime(df.calendar_last_scraped)
df.first_review = pd.to_datetime(df.first_review)
df.last_review = pd.to_datetime(df.last_review)
df.last_scraped = pd.to_datetime(df.last_scraped)

df.describe()

### Krok 5 - Podsumowanie
Zbudowanie podsumowania zmiennych **numerycznych** opisujących zbiór, w postaci jednej tabelki, zawierającej podstawowe informacje:

In [ ]:
desc_num = df.select_dtypes(include="number")                                               # wybranie tylko wartości numerycznych
desc_num = desc_num.describe()                                                              # opis podstawowych parametrów wybranych danych
desc_num = desc_num._append(desc_num.median(numeric_only=True).rename("median"))            # dodanie do tabeli mediany
desc_num = desc_num._append(desc_num.isna().sum().rename("missing"))                        # dodanie do tabeli ilości brakującyh danych

# desc_num = pd.concat([desc_num, desc_num.median(numeric_only=True).rename("median")])     # dodanie do tabeli mediany
# desc_num = pd.concat([desc_num, desc_num.isna().sum().rename("missing")])                 # dodanie do tabeli ilości brakującyh danych

desc_num

Analiza zmiennych **kategorycznych**:

In [ ]:
desc_cat = df.select_dtypes(include="category")                                                             # wybranie zmiennych kategorycznych
desc_cat = desc_cat.describe()                                                                              # opis podstawowych parametrów wybranych danych
desc_cat = desc_cat._append(desc_cat.loc["count"].subtract(df.shape[0]).abs().rename("missing"))            # dodanie ilości brakujących danych

# desc_cat = pd.concat([desc_cat, desc_cat.loc["count"].subtract(df.shape[0]).abs().rename("missing")])     # dodanie ilości brakujących danych

desc_cat

### Krok 6 - Sprawdzenie występowania braków danych w zbiorze

In [ ]:
df.isnull().sum()

In [ ]:
braki_danych = df.isnull().sum()

podsumowanie_brakow = pd.DataFrame({
    'Liczba_wystepowan': braki_danych.values,
    'Procent_wystepowan': (braki_danych / len(df)) * 100
})

print(podsumowanie_brakow)

### Krok 7 - Wizualizacja
Wizualizacja rozkładu (wybranych) zmiennych (zarówno numerycznych, jak i kategorycznych) poprzez histogramy i próba ich scharakteryzowania (np. poprzez ich skośność i kurtozę) – będzie to pomocne np. w procesie imputacji (uzupełniania) zmiennych numerycznych

### Wizualizacja zmiennych numerycznych:

In [ ]:
var_numeric = df.select_dtypes("number").isnull().sum()
var_numeric = var_numeric[var_numeric > 0]

for x in var_numeric.index:
    if df[x].nunique() < 120:
        try:
            print("\n\n wykres dla " + str(x) + ": ")
            plt.clf()
            df[x].groupby(df[x].values).sum().plot.bar(figsize=(30, 15), fontsize=14)
            plt.show()
        except:
            print("\t\tbłąd! (pusta kolumna): " + str(x))
            plt.clf()
    else:
        print("\n\n wykres dla " + str(x) + ": ")
        print("za duzo wartosci, zaokraglono do 0,1 \n ")
        plt.clf()
        zfile = round(df[x], 1)
        zfile.groupby(zfile.values).sum().plot.bar(figsize=(30, 15), fontsize=14)
        plt.show()

### Wizualizacja zmiennych kategorycznych: 

In [ ]:
var_categoric = df.select_dtypes("category").isnull().sum()
var_categoric = var_categoric[var_categoric > 0]

for x in var_categoric.index:
    if df[x].nunique() < 120:
        try:
            print("\n\n wykres dla " + str(x) + ": ")
            plt.clf()
            df[x].value_counts().plot.bar(figsize=(30, 15), fontsize=14)
            plt.show()
        except:
            print("\t\tbłąd! (pusta kolumna): " + str(x))
            plt.clf()
    else:
        print("\n\n wykres dla " + str(x) + ": ")
        plt.clf()
        df[x].value_counts().plot.bar(figsize=(60, 20), fontsize=12)
        plt.show()

### Charakteryzacja zmiennych:

In [ ]:
from scipy.stats import skew, kurtosis

# wybrane zmienne numeryczne
zmienne_numeryczne = ['bathrooms', 'bedrooms', 'beds', 'price']

# charakteryzacja zmiennych numerycznych
for it in zmienne_numeryczne:
    print(f'{it}:')
    print(f' Skośność: {skew(df[it]):.2f}')
    print(f' Kurtoza: {kurtosis(df[it]):.2f}')

### Krok 8 - Czyszczenie danych
Uzupełnienie brakujących danych (numeryczne – wartością średnią, kategoryczne – interpolacja)

In [ ]:
# wartosci numeryczne – uzupelnienie wartoscia srednia
for x in var_numeric.index:
    df[x].fillna((df[x].mean()), inplace=True)

# wartosci kategoryczne – interpolacja
for x in var_categoric.index:
    df[x] = df[x].interpolate(method='pad')

# usuwanie pustych kolumn
df.dropna(how='all', axis='columns', inplace=True)

### Krok 9 - Zbadanie zależności pomiędzy zmiennymi
### Obliczenie macierzy korelacji za pomocą współczynnika Pearsona

In [ ]:
cor_numeric = df.select_dtypes("number")
correlations = cor_numeric.corr(method='pearson', min_periods=1)
correlations

### Zwizualizowanie za pomocą Scatter Plot, Pair Plot i Heat Map

In [ ]:
import seaborn as sns
sns.set()

plt.figure(figsize=(20,30))
sns.scatterplot(correlations, alpha=0.5)
plt.xticks(rotation=45)
plt.title('Scatter Plot')
plt.show()

# plt.figure(figsize=(10,10))
# sns.pairplot(correlations)
# plt.title('Pair Plot')
# plt.show()

plt.figure(figsize=(60,60))
sns.heatmap(correlations, annot=True, cmap='vlag', vmin=-1, vmax=1)
plt.title('Heat Map')
plt.show()